# Volume-Based Estimators

This concept was originally developed by <strong data-cite="Lucy1999">[]</strong> and successively refined, for example, by <strong data-cite="Lucy1999a">[]</strong>, <strong data-cite="Lucy2002">[]</strong> and <strong data-cite="Lucy2003">[]</strong>.

When TARDIS runs, we enter into a loop with two main components: a Monte Carlo iteration occurs, and then the plasma state is updated based on the estimators.

## Theory

Ordinarily, TARDIS is not concerned about the physical amount of time a packet spends traveling through the ejecta. Instead, we consider the "time of simulation" $\Delta t$ which is chosen to be the amount of time in which the photosphere emmits the ensemble of packets (see [Energy Packet Initialization](../montecarlo/initialization.ipynb)). When looking at the estimators, a slightly different interpretation of the packets is necessary. Here, we view the packets as not carrying a discrete amount of energy $\varepsilon$ that is emitted in a time interval $\Delta t$, but as being a flow of energy that carries an energy $\varepsilon$ over a time $\Delta t$ -- that is, each packet is carrying a luminosity (energy per unit time) of $L = \frac{\varepsilon}{\Delta t}$. Now, we can say that if a packet spends a time $\delta t$ in the supernova's ejecta, it contributes an energy of $L\delta t= \frac{\varepsilon}{\Delta t}\delta t$ into the radiation energy of the ejecta.

To account for the effects of the Monte Carlo packets on the ejecta, TARDIS uses the packets to first determine the average radiation energy density $E$ throughout each shell, where the energy density is the total radiation energy in the shell divided by the volume of the shell ($\Delta V=4\pi (r_\mathrm{outer}^3-r_\mathrm{inner}^3)$). Therefore, we add up the amount of energy each packet contributes to the radiation energy in that shell, and divide by the total volume of the shell:
$$E=\frac{1}{\Delta V}\sum L_i\delta t_i=\frac{1}{\Delta V}\sum \frac{\varepsilon_i}{\Delta t}\delta t_i = \frac{1}{\Delta V\Delta t}\sum \varepsilon_i\delta t_i$$
where we sum over every Monte Carlo packet in the shell. Note that we are interested in the energy density in the co-moving frame (i.e. the energy density "according to the plasma," see REFERENCE FRAMES!!!!!!!!!!!!) Now, we note that the amount of time the Monte Carlo packet spends in a shell is $\delta t = \frac{l_i}{c}$ where $l$ is the distance that the packet travels through the shell. Thus, our estimator is
$$E=\frac{1}{\Delta V\Delta t}\sum \varepsilon_i\frac{l_i}{c}D_i = \frac{1}{c\Delta V\Delta t}\sum \varepsilon_i l_iD_i.$$

Using this energy density, we can then calculate the mean radiation intensity in that shell using the relation $J=\frac{c}{4\pi} E$, which gives us
$$J=\frac{1}{4\pi\Delta V\Delta t}\sum \varepsilon_i l_iD_i.$$

Next, we calculate the mean radiation frequency in each shell. To do this we use the same procedure as with $J$, but in the sum EXPLAIN??????????????????. Thus, we have
$$\bar \nu = \frac{1}{4\pi \Delta V \Delta t}\sum_i \varepsilon_i \nu_i l_i D_i.$$


REREAD THIS FOR REFERENCE FRAME STUFF

## Implimentation

**Mention that each iteration is just running the model and storing the state, plus some other stuff for recording things etc. MENTION CONVERGENCE???**

## Line Estimators

section

talk about how J_blues are not always updated.

## Code Example

We now show a detailed example of how the plasma is updated using the estimators after a Monte Carlo iteration. First, we import the needed packages and set up a simulation (see [Setting Up the Simulation](../setup/index.rst)):

In [ ]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.io.atom_data.util import download_atom_data
import numpy as np

# We download the atomic data needed to run this notebook
download_atom_data('kurucz_cd23_chianti_H_He')

In [ ]:
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)

model = sim.model
plasma = sim.plasma
runner = sim.runner

We show the initial radiative temperature, dilution factor, electron densities, and tau sobolevs in each radial cell (see **link**):

In [ ]:
model.t_rad

In [ ]:
model.w

In [ ]:
plasma.get_value('electron_densities')

In [ ]:
plasma.get_value('tau_sobolevs')

We set the number of packets and we run one iteration of the Monte Carlo simulation:

In [ ]:
N_packets = 10000

# Using the commented out code below, we can also get the number of packets
# from the configuration -- try it out:
#N_packets = tardis_config.no_of_packets

sim.iterate(N_packets)

We now show the values of the three estimators previously mentioned:

In [ ]:
runner.j_estimator

In [ ]:
runner.nu_bar_estimator

In [ ]:
# Because most rows of the j_blue estimatior are partially or mostly
# zero, we show just rows with all nonzero elements
runner.j_blue_estimator[np.all(runner.j_blue_estimator != 0, axis=1)]

We note that the shape of the j_blue estimator and the tau_sobolevs are the same: namely, each contain a value for each possible atomic line transition in each radial cell (as opposed to the other two estimators which just have one value for each cell):

In [ ]:
plasma.get_value('tau_sobolevs').shape, runner.j_blue_estimator.shape

We now advance the state of the simulation based on the estimators, and demonstrate this by showing the four quantities we showed before running the simulation. Compare them with their values above!

In [ ]:
# When advance_state is called, a brief summary of the updated t_rad's
# and w's is given
sim.advance_state();

In [ ]:
model.t_rad

In [ ]:
model.w

In [ ]:
plasma.get_value('electron_densities')

In [ ]:
plasma.get_value('tau_sobolevs')

check what happens to the j_blues before and after advance state